# MoE-PEFT: An Efficient LLM Fine-Tuning Factory for Mixture of Expert (MoE) Parameter-Efficient Fine-Tuning.
[![](https://github.com/TUDB-Labs/MoE-PEFT/actions/workflows/python-test.yml/badge.svg)](https://github.com/TUDB-Labs/MoE-PEFT/actions/workflows/python-test.yml)
[![](https://img.shields.io/github/stars/TUDB-Labs/MoE-PEFT?logo=GitHub&style=flat)](https://github.com/TUDB-Labs/MoE-PEFT/stargazers)
[![](https://img.shields.io/github/v/release/TUDB-Labs/MoE-PEFT?logo=Github)](https://github.com/TUDB-Labs/MoE-PEFT/releases/latest)
[![](https://img.shields.io/pypi/v/moe_peft?logo=pypi)](https://pypi.org/project/moe_peft/)
[![](https://img.shields.io/docker/v/mikecovlee/moe_peft?logo=Docker&label=docker)](https://hub.docker.com/r/mikecovlee/moe_peft/tags)
[![](https://img.shields.io/github/license/TUDB-Labs/MoE-PEFT)](http://www.apache.org/licenses/LICENSE-2.0)

MoE-PEFT is an open-source *LLMOps* framework built on [m-LoRA](https://github.com/TUDB-Labs/mLoRA). It is designed for high-throughput fine-tuning, evaluation, and inference of Large Language Models (LLMs) using techniques such as MoE + Others (like LoRA, DoRA). Key features of MoE-PEFT include:

- Concurrent fine-tuning, evaluation, and inference of multiple adapters with a shared pre-trained model.

- **MoE PEFT** optimization, mainly for [MixLoRA](https://github.com/TUDB-Labs/MixLoRA) and other MoLE implementation.

- Support for multiple PEFT algorithms and various pre-trained models.

- Seamless integration with the [HuggingFace](https://huggingface.co) ecosystem.

## About this notebook

This is a simple jupiter notebook for showcasing the basic process of building MixLoRA MoE model from **TinyLLaMA** by fine-tuning with dummy data.

## Clone and install MoE-PEFT

In [ ]:
! pip uninstall torchvision torchaudio -y
! pip install moe_peft

## Loading the base model

In [ ]:
import torch

import moe_peft

moe_peft.setup_logging("INFO")

base_model = "TinyLlama/TinyLlama_v1.1"

model = moe_peft.LLMModel.from_pretrained(
    base_model,
    device=moe_peft.executor.default_device_name(),
    load_dtype=torch.bfloat16,
)
tokenizer = moe_peft.Tokenizer(base_model)

## Training a dummy LoRA adapter

In [ ]:
lora_config = moe_peft.adapter_factory(
    peft_type="MIXLORA",
    adapter_name="mixlora_0",
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    target_modules=[
        "up_proj",
        "down_proj",
        "gate_proj",
    ],
    routing_strategy="mixlora",
    num_experts=6,
)

model.init_adapter(lora_config)

train_config = moe_peft.TrainConfig(
    adapter_name="mixlora_0",
    data_path="TUDB-Labs/Dummy-MoE-PEFT",
    num_epochs=10,
    batch_size=16,
    micro_batch_size=8,
    learning_rate=1e-4,
)

moe_peft.train(model=model, tokenizer=tokenizer, configs=[train_config])

## Validate the effectiveness of LoRA adapter

In [ ]:
generate_config = moe_peft.GenerateConfig(
    adapter_name="mixlora_0",
    prompts=["Could you provide an introduction to MoE-PEFT?"],
    stop_token="\n",
)

output = moe_peft.generate(
    model=model, tokenizer=tokenizer, configs=[generate_config], max_gen_len=128
)

print(output["mixlora_0"][0])